In [ ]:
from collections import defaultdict
import os
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.weight_estimator import WeightEstimator, CameraMetadata
from matplotlib import pyplot as plt
import pandas as pd

s3 = S3AccessUtils('/root/data')
RDS = RDSAccessUtils()

<h1> Rectification Library </h1>

In [ ]:
import json
from typing import Dict
import cv2
import numpy as np


IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000


def get_camera_parameters(stereo_params: Dict, 
                          new_camera_parameters_1=None, 
                          new_camera_parameters_2=None, 
                          new_camera_intrinsics_1=None,
                          new_camera_intrinsics_2=None,
                          new_R=None, 
                          new_T=None,
                          new_rd_1=None, 
                          new_rd_2=None,
                          new_pp_1=None,
                          new_pp_2=None) -> Dict:
    """Return individual camera parameters from JSON stereo parameters contents."""

    params = dict(stereo_params)
    if new_camera_parameters_1:
        params['CameraParameters1'] = new_camera_parameters_1
    if new_camera_parameters_2:
        params['CameraParameters2'] = new_camera_parameters_2
    if new_camera_intrinsics_1:
        params['CameraParameters1']['IntrinsicMatrix'] = new_camera_intrinsics_1
    if new_camera_intrinsics_2:
        params['CameraParameters2']['IntrinsicMatrix'] = new_camera_intrinsics_2
    if new_R:
        params['RotationOfCamera2'] = new_R
    if new_T:
        params['TranslationOfCamera2'] = new_T
    if new_rd_1:
        params['CameraParameters1']['RadialDistortion'] = new_rd_1
    if new_rd_2:
        params['CameraParameters2']['RadialDistortion'] = new_rd_2
    if new_pp_1:
        params['CameraParameters1']['PrincipalPoint'] = new_pp_1
    if new_pp_2:
        params['CameraParameters1']['PrincipalPoint'] = new_pp_2
        
    
    camera_matrix_1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    camera_matrix_2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    dist_coeffs_1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                    params['CameraParameters1']['TangentialDistortion'] + \
                    [params['CameraParameters1']['RadialDistortion'][2]]
    dist_coeffs_1 = np.array(dist_coeffs_1)

    dist_coeffs_2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                    params['CameraParameters2']['TangentialDistortion'] + \
                    [params['CameraParameters2']['RadialDistortion'][2]]
    dist_coeffs_2 = np.array(dist_coeffs_2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(camera_matrix_1, dist_coeffs_1,
                                                               camera_matrix_2,
                                                               dist_coeffs_2, image_size, R, T,
                                                               None,
                                                               None,
                                                               None, None, None,
                                                               cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(camera_matrix_1, dist_coeffs_1, R1, P1, image_size,
                                            cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(camera_matrix_2, dist_coeffs_2, R2, P2, image_size,
                                             cv2.CV_16SC2)

    params = {
        'left_maps': left_maps,
        'right_maps': right_maps,
        'camera_matrix_1': camera_matrix_1,
        'dist_coeffs_1': dist_coeffs_1,
        'R1': R1,
        'P1': P1,
        'camera_matrix_2': camera_matrix_2,
        'dist_coeffs_2': dist_coeffs_2,
        'R2': R2,
        'P2': P2
    }
    return params


def unrectify(ann: Dict, params: Dict):
    """Un-rectify ann with params."""

    left_maps = params['left_maps']
    right_maps = params['right_maps']

    ann_u = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                y_new, x_new = left_maps[0][y, x]
            elif side == 'rightCrop':
                y_new, x_new = right_maps[0][y, x]
            else:
                raise Exception('Invalid side!')

            ann_u[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_u


def rectify(ann: Dict, params: Dict) -> Dict:
    """Rectify ann with params."""

    camera_matrix_1 = params['camera_matrix_1']
    dist_coeffs_1 = params['dist_coeffs_1']
    R1 = params['R1']
    P1 = params['P1']

    camera_matrix_2 = params['camera_matrix_2']
    dist_coeffs_2 = params['dist_coeffs_2']
    R2 = params['R2']
    P2 = params['P2']

    ann_r = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[y, x]]).astype(float),
                        camera_matrix_1,
                        dist_coeffs_1,
                        R=R1,
                        P=P1)[0][0]
            elif side == 'rightCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[y, x]]).astype(float),
                        camera_matrix_2,
                        dist_coeffs_2,
                        R=R2,
                        P=P2)[0][0]
            else:
                raise Exception('Invalid side!')

            ann_r[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_r


def un_re_rectify(df, params_o, params_n):

    ann_u_rs = []
    for idx, row in df.iterrows():
        ann = row.annotation
        if ann is None:
            ann_u_rs.append(None)
            continue

        # un-rectify with original params
        ann_u = unrectify(ann, params_o)
        if ann_u:

            # re-rectify with new params
            ann_u_r = rectify(ann_u, params_n)
            ann_u_rs.append(ann_u_r)
        else:
            ann_u_rs.append(None)

    df['ann_u_r'] = ann_u_rs

In [ ]:
def get_camera_metadata(stereo_params, new_focal_length=None, new_translation_of_camera_2=None):
    stereo_parameters = dict(stereo_params)
    if new_focal_length:
        stereo_parameters['CameraParameters1']['FocalLength'] = new_focal_length
    if new_translation_of_camera_2:
        stereo_parameters['TranslationOfCamera2'] = new_translation_of_camera_2
    
    camera_metadata = {
        'focalLengthPixel': stereo_parameters['CameraParameters1']['FocalLength'][0],
        'baseline': abs(stereo_parameters['TranslationOfCamera2'][0] / 1e3),
        'focalLength': stereo_parameters['CameraParameters1']['FocalLength'][0] * 3.45e-6,
        'pixelCountWidth': 4096,
        'pixelCountHeight': 3000,
        'imageSensorWidth': 0.01412,
        'imageSensorHeight': 0.01035
    }
    
    return camera_metadata


def add_weights_u_r(df, camera_metadata):
    
    weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
    kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

    pred_weights = []
    for idx, row in df.iterrows():
        ann = row.ann_u_r
        if ann is not None:
            cm = CameraMetadata(
                focal_length=camera_metadata['focalLength'],
                focal_length_pixel=camera_metadata['focalLengthPixel'],
                baseline_m=camera_metadata['baseline'],
                pixel_count_width=camera_metadata['pixelCountWidth'],
                pixel_count_height=camera_metadata['pixelCountHeight'],
                image_sensor_width=camera_metadata['imageSensorWidth'],
                image_sensor_height=camera_metadata['imageSensorHeight']
            )

            weight, _, _ = weight_estimator.predict(ann, cm)
            pred_weights.append(weight)
        else:
            pred_weights.append(None)
    
    df['weight_u_r'] = pred_weights

<h1> Load dataset </h1>

In [ ]:
def extract_biomass_data(pen_id, start_date, end_date, akpd_score_cutoff):
    """Get raw biomass computations for given pen_id, date range, and AKPD score cutoff."""

    query = """
        SELECT * FROM
        prod.biomass_computations bc
        WHERE bc.pen_id={}
        AND bc.akpd_score >= {}
        AND bc.captured_at BETWEEN '{}' and '{}'
        AND bc.estimated_weight_g > 0.0
    """.format(pen_id, akpd_score_cutoff, start_date, end_date)

    df = RDS.extract_from_database(query)
    return df

In [ ]:
pen_id = 145
start_date = '2021-02-01'
end_date = '2021-02-05'
akpd_score_cutoff = 0.95
df = extract_biomass_data(pen_id, start_date, end_date, akpd_score_cutoff)

<h1> Un- and Re-rectify key-points </h1>

In [ ]:
stereo_params_o['CameraParameters2']

In [ ]:
stereo_params_n['CameraParameters2']

In [ ]:
stereo_params_o['CameraParameters1']

In [ ]:
stereo_params_n['CameraParameters1']

In [ ]:
def construct_new_stereo_parameters(stereo_parameters_o, stereo_parameters_n, 
                                    change_PP_1=False, change_PP_2=False, 
                                    change_focal_length_1=False, change_focal_length_2=False,
                                    change_rd_1=False, change_rd_2=False, 
                                    change_R=False, change_T=False):
    
    stereo_parameters_new = dict(stereo_parameters_o)
    
    if change_PP_1:
        stereo_parameters_new['CameraParameters1']['PrincipalPoint'] = \
            stereo_parameters_n['CameraParameters1']['PrincipalPoint']
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][2][:2] = \
            stereo_parameters_n['CameraParameters1']['PrincipalPoint']
    if change_PP_2:
        stereo_parameters_new['CameraParameters2']['PrincipalPoint'] = \
            stereo_parameters_n['CameraParameters2']['PrincipalPoint']
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][2][:2] = \
            stereo_parameters_n['CameraParameters2']['PrincipalPoint']
    if change_focal_length_1:
        stereo_parameters_new['CameraParameters1']['FocalLength'] = \
            stereo_parameters_n['CameraParameters1']['FocalLength']
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][0][0] = \
            stereo_parameters_n['CameraParameters1']['FocalLength'][0]
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][1][1] = \
            stereo_parameters_n['CameraParameters1']['FocalLength'][1]
    if change_focal_length_2:
        stereo_parameters_new['CameraParameters2']['FocalLength'] = \
            stereo_parameters_n['CameraParameters2']['FocalLength']
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][0][0] = \
            stereo_parameters_n['CameraParameters2']['FocalLength'][0]
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][1][1] = \
            stereo_parameters_n['CameraParameters2']['FocalLength'][1]
    if change_rd_1:
        stereo_parameters_new['CameraParameters1']['RadialDistortion'] = \
            stereo_parameters_n['CameraParameters1']['RadialDistortion']
    if change_rd_2:
        stereo_parameters_new['CameraParameters2']['RadialDistortion'] = \
            stereo_parameters_n['CameraParameters2']['RadialDistortion']
    if change_R:
        stereo_parameters_new['RotationOfCamera2'] = stereo_parameters_n['RotationOfCamera2']
    if change_T:
        stereo_parameters_new['TranslationOfCamera2'] = stereo_parameters_n['TranslationOfCamera2']
        
    return stereo_parameters_new

In [ ]:
# stereo_parameters_o_s3_url = 's3://aquabyte-stereo-parameters/L40076048_R40076044/2021-01-07T12:17:15.547471000Z_L40076048_R40076044_stereo-parameters.json'
stereo_parameters_o_s3_url = 's3://aquabyte-stereo-parameters/L40029797_R40020184/2020-10-05T22:39:45.664664000Z_L40029797_R40020184_stereo-parameters.json'
stereo_parameters_o_components = stereo_parameters_o_s3_url.replace('s3://', '').split('/')
bucket = stereo_parameters_o_components[0]
key = os.path.join(*stereo_parameters_o_components[1:])
stereo_parameters_o_f = s3.download_from_s3(bucket, key)

# stereo_parameters_n_s3_url = 's3://aquabyte-stereo-parameters/L40076048_R40076044/2021-03-10T14:15:41.082425000Z_L40076048_R40076044_stereo-parameters.json'
stereo_parameters_n_s3_url = 's3://aquabyte-stereo-parameters/L40029797_R40020184/2021-02-25T11:30:42.149694000Z_L40029797_R40020184_stereo-parameters.json'
stereo_parameters_n_components = stereo_parameters_n_s3_url.replace('s3://', '').split('/')
bucket = stereo_parameters_n_components[0]
key = os.path.join(*stereo_parameters_n_components[1:])
stereo_parameters_n_f = s3.download_from_s3(bucket, key)
stereo_params_o = json.load(open(stereo_parameters_o_f))
stereo_params_new = json.load(open(stereo_parameters_n_f))

In [ ]:
params_o = get_camera_parameters(stereo_params_o)
# params_n = get_camera_parameters(stereo_params_n)
params_n = get_camera_parameters(stereo_params_o, 
                                 new_camera_parameters_1=stereo_params_n['CameraParameters1'],
                                 new_camera_parameters_2=stereo_params_n['CameraParameters2'])

un_re_rectify(df, params_o, params_n)
cm = get_camera_metadata(stereo_params_o, new_focal_length=stereo_params_n['CameraParameters2']['FocalLength'])
add_weights_u_r(df, cm)
print(((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean())


In [ ]:
[False]

In [ ]:
analysis_data = defaultdict(list)
parameter_names = ['PP_1', 'PP_2', 'F1', 'F2', 'RD1', 'RD2', 'R', 'T']

for idx in range(8):
    bool_array = [False] * 8
    bool_array[idx] = True

#     stereo_parameters_o_s3_url = 's3://aquabyte-stereo-parameters/L40076048_R40076044/2021-01-07T12:17:15.547471000Z_L40076048_R40076044_stereo-parameters.json'
    stereo_parameters_o_s3_url = 's3://aquabyte-stereo-parameters/L40029797_R40020184/2020-10-05T22:39:45.664664000Z_L40029797_R40020184_stereo-parameters.json'
    stereo_parameters_o_components = stereo_parameters_o_s3_url.replace('s3://', '').split('/')
    bucket = stereo_parameters_o_components[0]
    key = os.path.join(*stereo_parameters_o_components[1:])
    stereo_parameters_o_f = s3.download_from_s3(bucket, key)

#     stereo_parameters_n_s3_url = 's3://aquabyte-stereo-parameters/L40076048_R40076044/2021-03-10T14:15:41.082425000Z_L40076048_R40076044_stereo-parameters.json'
    stereo_parameters_n_s3_url = 's3://aquabyte-stereo-parameters/L40029797_R40020184/2021-02-25T11:30:42.149694000Z_L40029797_R40020184_stereo-parameters.json'
    stereo_parameters_n_components = stereo_parameters_n_s3_url.replace('s3://', '').split('/')
    bucket = stereo_parameters_n_components[0]
    key = os.path.join(*stereo_parameters_n_components[1:])
    stereo_parameters_n_f = s3.download_from_s3(bucket, key)
    stereo_params_o = json.load(open(stereo_parameters_o_f))
    stereo_params_new = json.load(open(stereo_parameters_n_f))

    params_o = get_camera_parameters(stereo_params_o)
#     stereo_params_n = construct_new_stereo_parameters(stereo_params_o, 
#                                                       stereo_params_new, 
#                                                       change_PP_1=bool_array[0],
#                                                       change_PP_2=bool_array[1],
#                                                       change_focal_length_1=bool_array[2],
#                                                       change_focal_length_2=bool_array[3],
#                                                       change_rd_1=bool_array[4],
#                                                       change_rd_2=bool_array[5],
#                                                       change_R=bool_array[6],
#                                                       change_T=bool_array[7]
#                                                       )
    stereo_params_n = construct_new_stereo_parameters(stereo_params_o, 
                                                      stereo_params_new, 
                                                      change_PP_1=True,
                                                      change_PP_2=True,
                                                      change_focal_length_1=True,
                                                      change_focal_length_2=True,
                                                      change_rd_1=True,
                                                      change_rd_2=True,
                                                      change_R=True,
                                                      change_T=True
                                                      )

    params_n = get_camera_parameters(stereo_params_n)
    un_re_rectify(df, params_o, params_n)
    cm = get_camera_metadata(stereo_params_n)
    add_weights_u_r(df, cm)
    
    analysis_data['name'].append(parameter_names[idx])
    analysis_data['s1_value'].append(df.estimated_weight_g.mean())
    analysis_data['s1_prime_value'].append(df.weight_u_r.mean())
    analysis_data['pct_diff'].append(((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean())

    break

In [ ]:
analysis_df = pd.DataFrame(analysis_data)

In [ ]:
analysis_df.pct_diff.sum()

In [ ]:
analysis_df


In [ ]:
analysis_df


In [ ]:
stereo_params_o['CameraParameters1']['PrincipalPoint']

In [ ]:
stereo_params_new['CameraParameters1']['PrincipalPoint']

In [ ]:
stereo_params_o['CameraParameters2']['PrincipalPoint']

In [ ]:
stereo_params_new['CameraParameters2']['PrincipalPoint']

In [ ]:
stereo_params_o['CameraParameters1']['FocalLength']

In [ ]:
stereo_params_new['CameraParameters1']['FocalLength']

In [ ]:
stereo_params_o['CameraParameters2']['FocalLength']

In [ ]:
stereo_params_new['CameraParameters2']['FocalLength']

In [ ]:
stereo_params_o['CameraParameters1']['RadialDistortion']

In [ ]:
stereo_params_new['CameraParameters1']['RadialDistortion']

In [ ]:
stereo_params_o['CameraParameters2']['RadialDistortion']

In [ ]:
stereo_params_new['CameraParameters2']['RadialDistortion']

In [ ]:
stereo_params_o['RotationOfCamera2']

In [ ]:
stereo_params_new['RotationOfCamera2']

In [ ]:
stereo_params_o['TranslationOfCamera2']

In [ ]:
stereo_params_new['TranslationOfCamera2']

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()

In [ ]:
((df.weight_u_r - df.estimated_weight_g) / df.estimated_weight_g).mean()